In [22]:
import pandas as pd
import numpy as np

In [23]:
data = pd.read_csv("../data/cleaned_data.csv")
data.head()

,Unnamed: 0,date,ron95,ron97,diesel,ron95_budi95,USD,index,state,division,Rate
0,2000,2017-04-01,2.13,2.41,2.11,2.13,4.4265,122.800000,Johor,overall,3.0
1,2001,2017-04-01,2.13,2.41,2.11,2.13,4.4265,118.600000,Kedah,overall,3.0
2,2002,2017-04-01,2.13,2.41,2.11,2.13,4.4265,118.724922,Kelantan,overall,3.0
3,2003,2017-04-01,2.13,2.41,2.11,2.13,4.4265,119.100000,Melaka,overall,3.0
4,2004,2017-04-01,2.13,2.41,2.11,2.13,4.4265,120.000000,Negeri Sembilan,overall,3.0


# 1. CFPI (Consumer Financial Pressure Index) DESIGN


CFPI is a composite early-warning indicator that measures the level and momentum of financial pressure experienced by consumers, using macroeconomic cost and policy signals.

### a. Configuration of CFPI

In [24]:

ROLLING_Z_WINDOW = 24        # months
SMOOTHING_WINDOW = 3         # months

WEIGHTS = {
    "fuel": 0.30,
    "inflation": 0.35,
    "fx": 0.20,
    "policy": 0.15
}

CFPI_MIN = 0
CFPI_MAX = 100


### b. Feature Engineering

In [25]:


def rolling_zscore(series, window):
    mean = series.rolling(window).mean()
    std = series.rolling(window).std()
    return (series - mean) / std


def compute_fuel_pressure(df):
    fuel_price = (
        0.5 * df["ron95_budi95"] +
        0.3 * df["diesel"] +
        0.2 * df["ron97"]
    )

    fuel_mom_change = fuel_price.pct_change()
    fuel_pressure = rolling_zscore(fuel_mom_change, window=6)

    return fuel_pressure



def compute_inflation_pressure(df):
    cpi_change = df["index"].pct_change(periods=12)  # YoY inflation
    inflation_pressure = rolling_zscore(cpi_change, window=6)

    return inflation_pressure


def compute_fx_pressure(df):
    fx = df["USD"]
    fx_change_3m = fx.pct_change(periods=3)

    fx_pressure = rolling_zscore(fx_change_3m, window=6)
    return fx_pressure


def compute_policy_pressure(df):
    opr_change = df["Rate"].diff()

    policy_pressure = (opr_change > 0).astype(int)
    return policy_pressure


### c. Index construction logic (Calculation)

In [26]:

def scale_to_100(series):
    min_val = series.min()
    max_val = series.max()
    return 100 * (series - min_val) / (max_val - min_val)


def compute_cfpi(df):
    df = df.sort_values("date").copy()

    df["fuel_pressure"] = compute_fuel_pressure(df)
    df["inflation_pressure"] = compute_inflation_pressure(df)
    df["fx_pressure"] = compute_fx_pressure(df)
    df["policy_pressure"] = compute_policy_pressure(df)

    df["raw_cfpi"] = (
        WEIGHTS["fuel"] * df["fuel_pressure"] +
        WEIGHTS["inflation"] * df["inflation_pressure"] +
        WEIGHTS["fx"] * df["fx_pressure"] +
        WEIGHTS["policy"] * df["policy_pressure"]
    )

    df["raw_cfpi"] = df["raw_cfpi"].rolling(SMOOTHING_WINDOW, min_periods=1).mean()

    df["cfpi"] = scale_to_100(df["raw_cfpi"])
    df["cfpi"] = df["cfpi"].clip(CFPI_MIN, CFPI_MAX)

    df["cfpi_trend"] = df["cfpi"].diff()

    df["risk_flag"] = pd.cut(
        df["cfpi"],
        bins=[-np.inf, 30, 60, 80, np.inf],
        labels=["Low", "Moderate", "High", "Severe"]
    )

    return df


### d. run function

In [27]:


INPUT_PATH = "../data/cleaned_data.csv"
OUTPUT_PATH = "outputs/cfpi_daily.csv"

df = pd.read_csv(INPUT_PATH, parse_dates=["date"])

result = (
    df
    .groupby("state", group_keys=False)
    .apply(compute_cfpi)
    .reset_index(drop=True)
)

result.head()

result.to_csv("../data/data.csv")

C:\Users\dania\AppData\Local\Temp\ipykernel_7992\3135325127.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_cfpi)
